# Resultados y experimentos

En este notebook mostraremos algunos resultados estadísticos de la simulación

In [1]:
from simulation_engine import *

simulation_time:int = 14 * 60 # 14 hours * 60 min = 840
# The simulation time is in minutes
sim = InventorySimulation(sim_duration=simulation_time)
sim.run()

# registry = sim.registry

# registry.balance_registry

In [3]:
from sim_stats import FlattenRegistry, SimStatistics

sim_statistics = SimStatistics(sim.registry)

# plotting the seels of the store
sim_statistics.plot_sells()

AttributeError: 'SimStatistics' object has no attribute 'plot_sales'